In [1]:
import numpy as np
import os
import torch
import timm
import torch.nn as nn
from transformers import Wav2Vec2Processor, HubertModel
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_video
import pandas as pd
import time
from tqdm import tqdm
from sklearn.metrics import f1_score
import torchaudio
from moviepy.editor import VideoFileClip, vfx
import logging
import math
logging.getLogger('moviepy').setLevel(logging.ERROR)

/home1/bipengwa/.conda/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torchaudio.set_audio_backend("sox_io")  # 或者 "sox_io"
defalut_dtype = torch.float16

In [3]:
torch.cuda.is_available()

True

In [4]:
def create_transform():
    #transform image to image feature
    return transforms.Compose([
        transforms.ToPILImage(),  # 将 numpy 数组或 tensor 转换为 PIL 图像
        transforms.Resize((224, 224)),  # 调整图像大小
        transforms.ToTensor(),  # 将 PIL 图像转换为 tensor，并归一化至 [0,1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
    ])

def audio_pre(audio, info):
    """
    input:
            audio preprocessed by Wav2Vec2Processor
    return:
            after channel and fps adjusted audio
    """
    if audio.shape[0] > 1:  # More than one channel
        audio = torch.mean(audio, dim=0, keepdim=True)

    if info["audio_fps"] != 16000:
    # set sample rate to int(16000 * (0.02002 * info["video_fps"])), but
    # treat it as 16000, so that the audio speed is adjusted, in order to make
    # the generated feature length the same as number of frames
    # 0.02002 is a constant for HuBERT model
        resampler = torchaudio.transforms.Resample(orig_freq=info["audio_fps"],new_freq=int(16000 * (0.02002 * info["video_fps"])))
        resampled_audio = resampler(audio).squeeze()
    
    audio_feature = processor(resampled_audio, return_tensors="pt", sampling_rate=16000).input_values
    audio_feature = audio_feature.to(device)

    return audio_feature
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [5]:
video, audio, info = read_video("/project/msoleyma_1026/Aff-Wild2/video/326.mp4")

In [6]:
audio.shape

torch.Size([2, 8585216])

In [7]:
video.shape

torch.Size([4717, 360, 640, 3])

In [8]:
video[0].permute(2,0,1).shape

torch.Size([3, 360, 640])

In [9]:
info

{'video_fps': 24.227015921931176, 'audio_fps': 44100}

In [10]:
audio_feature = audio_pre(audio, info)

In [11]:
audio_feature.shape

torch.Size([1, 1510687])

In [107]:
class AudioVideoDataset(Dataset):
    def __init__(self, video_dir, label_dir, device, frame_inteval=5, dtype=torch.float32):
        self.video_dir = video_dir
        self.label_dir = label_dir
        self.device = device
        self.frame_inteval = frame_inteval
        self.dtype = dtype
        self.transform = self.create_transform()
        self.processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
        self.entries = self._collect_entries()

    def _collect_entries(self):
        possible_extensions = ['.mp4', '.avi']
        entries = []
        for label_file in sorted(os.listdir(self.label_dir)):
            if label_file.endswith('.txt'):
                base_name = os.path.splitext(label_file)[0]
                video_file = self.find_video_file(self.video_dir, base_name, possible_extensions)
                # print(os.path.exists(video_file) is None)
                if video_file:
                    entries.append((video_file, os.path.join(self.label_dir, label_file)))
        return entries

    def find_video_file(self, video_dir, base_name, possible_extensions):
        for ext in possible_extensions:
            video_path = os.path.join(video_dir, f"{base_name}{ext}")
            if os.path.exists(video_path):
                return video_path
            else:
                print(f"cannot find the video: {video_path}")
                return None

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        video_file_path, label_file_path = self.entries[idx]
        video, audio, info = read_video(video_file_path)

        # print(label_file_path)
        
        # Preprocess the audio first
        # print(audio.shape)
        audio_feature = self.audio_pre(audio, info)  # Assuming audio_pre is defined to handle this
        # print(audio_feature.shape)
        
        video_fps = info['video_fps']
        frame_interval = max(1, int(video_fps / self.frame_inteval))
        # print(audio_feature.shape, video.shape)
        # audio_duration = audio_feature.shape[1] // video.shape[0]
        audio_duration = math.ceil(audio_feature.shape[1] / video.shape[0])
        
        sampled_video_frames = []
        sampled_audio_frames = torch.empty((0,), dtype=self.dtype, device = self.device)  # Correct dtype
        # print(sampled_audio_frames.shape)
        labels = torch.tensor(np.loadtxt(label_file_path, skiprows=1, delimiter=','), dtype=self.dtype)
        sampled_labels = []

        for i in range(0, len(video), frame_interval):
            frame = video[i].permute(2, 0, 1)
            sampled_video_frames.append(self.transform(frame))
            sampled_labels.append(labels[min(i, len(labels) - 1)])  # Safeguard index
            
            # Extract corresponding audio segment
            start = i * audio_duration
            end = (i + 1) * audio_duration
            sampled_audio_segment = audio_feature[:, start:end]
            # print(sampled_audio_segment.shape)
            sampled_audio_frames = torch.cat((sampled_audio_frames, sampled_audio_segment), dim=1)
            # print(sampled_audio_frames.shape)
        
        video_feature = torch.stack(sampled_video_frames)
        labels_feature = torch.stack(sampled_labels)

         #video_feature 此时 torch.Size([num_frames, 3, 224, 224])， audio_feature还为raw， 经过hubert后，转为 torch.Size([1, num_frames, 1024])， 需要在后续模型中进行对齐

        return video_feature, sampled_audio_frames, labels_feature
        
    def create_transform(self):
        #transform image to image feature
        return transforms.Compose([
            transforms.ToPILImage(),  # 将 numpy 数组或 tensor 转换为 PIL 图像
            transforms.Resize((224, 224)),  # 调整图像大小
            transforms.ToTensor(),  # 将 PIL 图像转换为 tensor，并归一化至 [0,1]
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
        ])

    def audio_pre(self,audio, info):
        """
        input:
                audio preprocessed by Wav2Vec2Processor
        return:
                after channel and fps adjusted audio
        """
        if audio.shape[0] > 1:  # More than one channel
            audio = torch.mean(audio, dim=0, keepdim=True)

        if info["audio_fps"] != 16000:
        # set sample rate to int(16000 * (0.02002 * info["video_fps"])), but
        # treat it as 16000, so that the audio speed is adjusted, in order to make
        # the generated feature length the same as number of frames
        # 0.02002 is a constant for HuBERT model
            resampler = torchaudio.transforms.Resample(orig_freq=info["audio_fps"],new_freq=int(16000 * (0.02002 * info["video_fps"])))
            resampled_audio = resampler(audio).squeeze()
        
        audio_feature = self.processor(resampled_audio, return_tensors="pt", sampling_rate=16000).input_values
        audio_feature = audio_feature.to(device, dtype = self.dtype)

        return audio_feature
        

In [108]:
# test
#train_label_dir = "/project/msoleyma_1026/Aff-Wild2/labels/EXPR_Classification_Challenge/Train_Set"
#val_label_dir = "/project/msoleyma_1026/Aff-Wild2/labels/EXPR_Classification_Challenge/Validation_Set"
video_dir = "/project/msoleyma_1026/Aff-Wild2/video"

train_label_dir = "/project/msoleyma_1026/Aff-Wild2/test/test_3_expr/Train"
val_label_dir = "/project/msoleyma_1026/Aff-Wild2/test/test_3_expr/Val"

# train_dataset = AudioVideoDataset(video_dir, train_label_dir, device)
val_dataset = AudioVideoDataset(video_dir, val_label_dir, device)

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
cannot find the video: /project/msoleyma_1026/Aff-Wild2/video/135.mp4
cannot find the video: /project/msoleyma_1026/Aff-Wild2/video/136.mp4
cannot find the video: /project/msoleyma_1026/Aff-Wild2/video/137.mp4
cannot find the video: /project/msoleyma_1026/Aff-Wild2/video/147.mp4


In [109]:
os.path.exists('/project/msoleyma_1026/Aff-Wild2/video/135.mp4')

False

In [110]:
len(val_dataset)

2

In [111]:
for video,audio,label in val_dataset:
    print(video.shape, audio.shape, label.shape)
    break
    

torch.Size([369, 3, 224, 224]) torch.Size([1, 118449]) torch.Size([369])


In [15]:
# test = train_dataset[0]

In [24]:
class ViTHuBERTTransformer(nn.Module):
    def __init__(self, vit_base_model,
                 hubert_base_model,
                 num_classes,
                 nhead,
                 num_layers,
                 seq_len,
                small_dataset = False):
        super().__init__()

        self.seq_len = seq_len
        self.num_classes = num_classes
        self.vit = timm.create_model(vit_base_model, pretrained=True)

        #self.processor = Wav2Vec2Processor.from_pretrained(hubert_base_model)
        self.hubert = HubertModel.from_pretrained(hubert_base_model)

        if small_dataset:
            for param in self.vit.parameters():
                param.requires_grad = False
        
            for param in self.hubert.parameters():
                param.requires_grad = False
            

        encoder_layer = nn.TransformerEncoderLayer(d_model = self.vit.num_features + self.hubert.config.hidden_size,
                                                  nhead = nhead,
                                                  dim_feedforward = (self.vit.num_features + self.hubert.config.hidden_size)//2,
                                                  batch_first = True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers = num_layers)

        # Classifier
        self.classifier = nn.Linear(self.vit.num_features + self.hubert.config.hidden_size, num_classes)
    def forward(self, video_feature_raw, audio_feature_raw):

        audio_feature = self.hubert(audio_feature_raw).last_hidden_state
        vit_feature = self.vit.forward_features(video_feature_raw)
        # print(audio_feature.shape, vit_feature.shape)

        #print(audio_feature.shape)
        #print(vit_feature.shape)

        combined_features = self.seq_feature_generation(vit_feature, audio_feature, self.seq_len)
        # print(combined_features.shape, self.seq_len)
        #[batch, seq_len, combined_feature_size]

        #print(combined_features.shape)
        transformer_output = self.transformer_encoder(combined_features)
        
        logits = self.classifier(transformer_output)
        logits = logits.view(-1, self.num_classes)
        return logits

    def seq_feature_generation(self, video_feature, audio_feature, seq_len, pooling = "mean"):
        #video_feature : (771, 1, 197, 768)
        #audio_feature : [1, 773, 1024]
        video_feature = torch.tensor(video_feature, dtype=torch.float32).unsqueeze(1)
        audio_feature = torch.tensor(audio_feature, dtype=torch.float32)

        if audio_feature.shape[1] < video_feature.shape[0]:
            # 需要增加的长度
            padding_length = video_feature.shape[0] - audio_feature.shape[1]
            
            # 取最后一个时间步的特征进行复制
            last_frame_features = audio_feature[:, -1, :]
            
            # 重复最后一帧特征直到达到所需长度
            padding = last_frame_features.repeat(1, padding_length, 1)
            
            # 将原始音频特征和填充特征拼接起来
            padded_audio_feature = torch.cat([audio_feature, padding], dim=1)
        else:
            padded_audio_feature = audio_feature


        #print(padded_audio_feature.shape)

        video_feature = video_feature.permute(1,0,2,3)
        
        if pooling == "mean":
            video_feature = torch.mean(video_feature, dim = 2, keepdim=False)
        elif pooling == "max":
            video_feature = torch.max(video_feature, dim = 2, keepdim=False)[0]
    
        max_seq = min(video_feature.shape[1], padded_audio_feature.shape[1])
        video_feature = video_feature[:, :max_seq, :]
        audio_feature = padded_audio_feature[:, :max_seq, :]

        #print(video_feature.shape)
        #print(audio_feature.shape)
        
        combined_feature = torch.cat([video_feature, audio_feature], dim = -1)
        #[1, max_seq, 1024 + 768]

        #print(combined_feature.shape)
        
        if max_seq < seq_len:
            # Pad both features to seq_len along the sequence dimension
            combined_sequences = F.pad(combined_feature, (0, 0, 0, seq_len - max_seq))
        else:
            num_complete_seqs = max_seq // seq_len
            combined_sequences = combined_feature[:,:num_complete_seqs*seq_len, :].view(-1, seq_len, combined_feature.shape[-1])
        #[batch, seq_len, combined_feature_size]

        if combined_feature.shape[1]%self.seq_len !=0:
            tmp_seq = (combined_feature.shape[1]//self.seq_len)*self.seq_len
            combined_feature = combined_feature[:,:tmp_seq, :]
            
        return combined_sequences

In [25]:
model = ViTHuBERTTransformer(
    vit_base_model = 'vit_base_patch16_224',
    hubert_base_model = "facebook/hubert-large-ls960-ft",
    num_classes = 8,
    nhead = 8,
    num_layers = 6,
    seq_len = 10,
    small_dataset = False
)
#model = model.half()  # Convert all model parameters and buffers to float16
model.to(device)

ViTHuBERTTransformer(
  (vit): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='

In [159]:
for param in model.parameters():
    print(param.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [26]:
# model.to(device)

In [27]:
loss_function = torch.nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.96)

In [28]:
device

'cuda:0'

In [53]:
math.ceil(100 / 3)

34

In [54]:
math.ceil(100 // 3)

33

In [58]:
int(3.3)

3

In [59]:
int(3.7)

3

In [155]:
def train(model, optimizer, dataloader_train, dataloader_valid, loss_fn,
             seq_len = 10, batch_size = 3, max_iter=101, scheduler=None, device="cpu"):
    model.to(device = device, dtype=torch.float32)
    print(device)
    checkpoint_generator = loglinspace(0.3, 5)
    checkpoint = next(checkpoint_generator)
    start_time = time.time()
    run_name = "vithubertformer"
    try:
        model.load_state_dict(torch.load(run_name + '.torch')['state'])
    except:
        results = {}
        history = []
        s0 = 0
    else:
        results = torch.load(run_name + '.torch')
        history = results['history']
        s0 = history[-1]['step'] + 1

    for step in range(max_iter):
        model.train()
        loss_cumulative = 0.

        for j, d in tqdm(enumerate(dataloader_train), total=len(dataloader_train)):
            video_feature, audio_feature, labels = d
            
            video_feature = video_feature.squeeze(0)
            audio_feature = audio_feature.squeeze(0)
            labels = labels.squeeze(0).to(device)

            multiplicity = math.ceil(audio_feature.shape[1] / video_feature.shape[0])
            
            preparation_len = (video_feature.shape[0] // (seq_len * batch_size))*(seq_len * batch_size)
            video_feature = video_feature[:preparation_len]
            video_feature = video_feature.to(device)

            audio_feature = audio_feature[:,:preparation_len*multiplicity]
            audio_feature = audio_feature.to(device)
            
            # audio_duration = audio_feature.shape[1]//video_feature.shape[0]
            audio_duration = math.ceil(audio_feature.shape[1] / video_feature.shape[0])
            for i in range(0, len(video_feature), seq_len*batch_size):
                video_batch = video_feature[i:i + seq_len * batch_size]
                audio_batch = audio_feature[:,i*audio_duration:(i + seq_len * batch_size)*audio_duration]
                label_batch = labels[i:i + seq_len * batch_size]
                # print(i,audio_duration, video_feature.shape, audio_feature.shape, video_batch.shape, audio_batch.shape, label_batch.shape)
                

                #print(video_batch.shape)
                #print(audio_batch.shape)
                #print(label_batch.shape) 30
            #print(model.dtype)
                if label_batch.shape[0]%seq_len !=0:
                    tmp =  (label_batch.shape[0]//seq_len)*seq_len
                    label_batch = label_batch[:tmp]
                
                mask = (label_batch != -1)
            
                output = model(video_batch, audio_batch)
                #print(output.shape)  (2, 10, 7)

                filtered_output = output[mask]
                filtered_label_batch = label_batch[mask]
                if filtered_label_batch.shape[0] == 0:
                    continue
                # print(filtered_output.shape, filtered_label_batch.shape)
                #print(filtered_output.shape)
                #print(filtered_label_batch.shape)
                
                loss = loss_fn(filtered_output, filtered_label_batch.long()).cpu()
                loss_cumulative = loss_cumulative + loss.detach().item()
                # print(loss, loss_cumulative)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        end_time = time.time()
        wall = end_time - start_time

        if step == checkpoint:
            checkpoint = next(checkpoint_generator)
            assert checkpoint > step

            valid_avg_loss = evaluate(model, dataloader_valid, seq_len, batch_size, loss_fn, device)
            # train_avg_loss = evaluate(model, dataloader_train, seq_len, batch_size, loss_fn, device)

            history.append({
                'step': s0 + step,
                'wall': wall,
                'batch': {
                    'loss': loss.item(),
                },
                'valid': {
                    'loss': valid_avg_loss,
                },
            })

            results = {
                'history': history,
                'state': model.state_dict()
            }

            print(f"epoch {step + 1:4d}   " +
                  f"abs = No train loss   " +
                  f"valid loss mse= {valid_avg_loss:8.4f}   " +
                  f"wall = {time.strftime('%H:%M:%S', time.gmtime(wall))}")

            with open(run_name + '.torch', 'wb') as f:
                torch.save(results, f)

        if scheduler is not None:
            scheduler.step()

def evaluate(model, dataloader, seq_len, batch_size, loss_fn, device):
    model.eval()
    loss_cumulative = 0.
    start_time = time.time()
    with torch.no_grad():
        for j, d in tqdm(enumerate(dataloader), total=len(dataloader)):
            video_feature, audio_feature, labels = d
            
            video_feature = video_feature.squeeze(0)
            audio_feature = audio_feature.squeeze(0)
            labels = labels.squeeze(0).to(device)

            multiplicity = math.ceil(audio_feature.shape[1] / video_feature.shape[0])
            
            preparation_len = (video_feature.shape[0] // (seq_len * batch_size))*(seq_len * batch_size)
            video_feature = video_feature[:preparation_len]
            video_feature = video_feature.to(device)

            audio_feature = audio_feature[:,:preparation_len*multiplicity]
            audio_feature = audio_feature.to(device)
            
            audio_duration = math.ceil(audio_feature.shape[1] / video_feature.shape[0])
            for i in range(0, len(video_feature), seq_len*batch_size):
                video_batch = video_feature[i:i + seq_len * batch_size]
                audio_batch = audio_feature[:,i*audio_duration:(i + seq_len * batch_size)*audio_duration]
                label_batch = labels[i:i + seq_len * batch_size]
            

                if label_batch.shape[0]%seq_len !=0:
                    tmp =  (label_batch.shape[0]//seq_len)*seq_len
                    label_batch = label_batch[:tmp]
                
                mask = (label_batch != -1)
            
                output = model(video_batch, audio_batch)
                
                filtered_output = output[mask]
                filtered_label_batch = label_batch[mask] 
                
                if filtered_label_batch.shape[0] == 0:
                    continue
                    
                loss = loss_fn(filtered_output, filtered_label_batch.long()).cpu()
                loss_cumulative = loss_cumulative + loss.detach().item()
                print(loss, loss_cumulative)
            #print(len(output))
            #print(len(d.target))

    return loss_cumulative / len(dataloader)

def loglinspace(rate, step, end=None):
    t = 0
    while end is None or t <= end:
        yield t
        t = int(t + 1 + step * (1 - math.exp(-t * rate / step)))

In [156]:
1031741//3222

320

In [157]:
377817//1180

320

In [158]:
# model.to(device = device, dtype=torch.float32)
train(model, opt, train_loader, val_loader, loss_function,max_iter=1, scheduler=scheduler, device=device)


cuda:0


  0%|          | 0/2 [00:00<?, ?it/s]

tensor(0.8178) 0.8177566528320312
tensor(7.7597) 8.577456951141357
tensor(11.8432) 20.420650959014893
tensor(11.8432) 32.26384496688843
tensor(11.8432) 44.10703897476196


 50%|█████     | 1/2 [00:09<00:09,  9.45s/it]

tensor(0.8178) 44.924795627593994
tensor(2.1821) 47.10687327384949
tensor(11.8432) 58.950069189071655
tensor(11.8432) 70.79326701164246
tensor(11.8432) 82.63646483421326
tensor(11.8432) 94.47966361045837
tensor(11.8432) 106.32286143302917
tensor(11.8432) 118.16605925559998
tensor(11.8432) 130.00925707817078
tensor(11.8432) 141.85245490074158
tensor(12.0798) 153.9322545528412
tensor(11.8432) 165.775452375412
tensor(11.8432) 177.6186501979828
tensor(11.8432) 189.4618480205536
tensor(11.8432) 201.3050458431244
tensor(11.8432) 213.1482436656952
tensor(11.8432) 224.991441488266
tensor(11.8432) 236.8346393108368
tensor(12.3558) 249.19048380851746
tensor(11.8432) 261.03368163108826
tensor(11.8432) 272.8768813610077
tensor(11.8432) 284.7200791835785
tensor(11.8432) 296.5632779598236
tensor(11.8432) 308.4064757823944
tensor(11.8432) 320.2496726512909
tensor(11.8432) 332.0928704738617
tensor(11.8432) 343.9360682964325
tensor(11.8432) 355.7792661190033


100%|██████████| 2/2 [00:24<00:00, 12.21s/it]

tensor(12.0469) 367.8261468410492
epoch    1   abs = No train loss   valid loss mse= 183.9131   wall = 00:01:34


In [ ]:
!nvidia-smi